# Tutorial on using Propnet

The following is a tutorial designed to give a base overview of the classes and constructs used in the Propnet project. For each class an example of its construction and base usage is provided.

# Defining a Property Network: Propnet

A Graph object tells us about all the property types and models currently available for use.

The mappings contained in this object define an interconnected network of materials properties. In this form, the Graph object can be used to enumerate and analyze links between differnet materials properties.

In [3]:
from propnet.core.graph import Graph

In [4]:
g = Graph()

You can print Propnet to see the property types and models it supports.

In [5]:
print(g)

Propnet Printout

Properties
	formula
	dimensionality
	structure_oxi
	oxide_type
	is_metallic
	magnetic_order
	external_identifier_mp
	computed_entry
	structure
	potentially_ferroelectric
	prototype
	composition
	null_symbol
	atomic_density
	elastic_tensor_voigt
	gruneisen_parameter
	interplanar_spacing
	sound_velocity_longitudinal
	goldschmidt_tolerance_factor
	thermal_conductivity
	refractive_index
	ionic_radius_b
	absorption_coefficient
	p_wave_modulus
	cost_per_kg
	extinction_coefficient
	electronic_thermal_conductivity
	pilling_bedworth_ratio
	transmittance
	interatomic_spacing
	band_gap_pbe
	electrical_resistivity
	relative_permittivity
	snyder_acoustic_sound_velocity
	cost_per_mol
	sound_velocity_mean
	volume_unit_cell
	youngs_modulus
	dielectric_figure_of_merit_energy
	mass_per_atom
	poisson_ratio
	absorbance
	specific_heat_capacity_constant_volume
	lame_first_parameter
	compliance_tensor_voigt
	hhi_reserve
	snyder_optical_sound_velocity
	reflectance
	formation_energy_per_atom


# Defining a Material Property

A Symbol object is used to represent types of properties (such as Young's Modulus) or conditions (such as Temperature).
- All Symbol objects are accessible in a global DEFAULT_SYMBOLS variable.
- Various metadata for each SymbolType can be accessed as shown below.

In [6]:
from propnet.symbols import DEFAULT_SYMBOLS
symbol_object = DEFAULT_SYMBOLS['youngs_modulus']
print(symbol_object)

Symbol: youngs_modulus


A Quantity object is used to represent values of properties (such as Young's Modulus = 200GPa) or conditions (such as temperature = 300K).

- All Quantity objects have a Symbol giving the type of property represented by the value.
- All Quantity objects must be created at runtime by specifying a value during instantiation.
- All Quantity objects have a list of strings called "tags" used to further label the property.

In [7]:
from propnet.core.quantity import Quantity
steel_youngs_modulus = Quantity('youngs_modulus', 200, tags=['mild steel'])
print(steel_youngs_modulus)

<youngs_modulus, 200 gigapascal, ['mild steel']>


# Defining a Material

A Material object is used to represent a collection of information known about a given material.

When it is first created it has no information; however, properties can be added to the material one-by-one.

In [8]:
from propnet.core.materials import Material
from propnet.core.quantity import Quantity
mild_steel = Material()
youngs_modulus = Quantity('youngs_modulus', 200, [])
mild_steel.add_quantity(youngs_modulus)
print(mild_steel)

Material: 0x1220a9ba8

	youngs_modulus
		<youngs_modulus, 200 gigapascal, None>



# Combining Models, Materials, and Symbols

As illustrated, a Graph object contains information for connecting many different models and symbol types. This forms an abstract web of interconnected variables without any values specified.

On the other hand, a Material object represents a grouping of values for different variables. These are represented as a collection of Quantity objects identified with the material.

At runtime, a single Graph object can be used to "evaluate" a Material object. This procedure allows values to be plugged in to variables. Assuming the required inputs for a model all have values, the Graph object can then dynamically predict the values for the output variables of the model.

In this example, we start with a refractive index and a permittivity. As a result many other properties are derived and printed below.

In [4]:
from propnet.core.materials import Material
from propnet.core.graph import Graph
from propnet.core.quantity import Quantity

g = Graph()

silica = Material()
refractive_index = Quantity('refractive_index', 1.458)
relative_permittivity = Quantity('relative_permittivity', 3.9)

silica.add_quantity(refractive_index)
silica.add_quantity(relative_permittivity)

silica = g.evaluate(silica)
print(silica)

Material: 0x1228d1f60

	refractive_index
		<refractive_index, 1.458 dimensionless, None>

	relative_permittivity
		<relative_permittivity, 3.9 dimensionless, None>

	relative_permeability
		...

	band_gap
		<band_gap, 21.02294283374089 electron_volt, None>
		<band_gap, 34.44429680416945 electron_volt, None>
		<band_gap, 8.44705399783503 electron_volt, None>

	dielectric_figure_of_merit_energy
		...
		...
		...

	band_gap_gw
		...
		...
		...

	band_gap_pbe
		...
		...
		...
		...
		...
		...

	dielectric_figure_of_merit_current_leakage
		...
		...
		...

	is_metallic
		<is_metallic, 0 dimensionless, None>



/Users/david/propnet_venv/lib/python3.6/site-packages/numpy/__init__.py:1: RuntimeWarning: invalid value encountered in sqrt
  """


# Working with Models

A Model object is used to represent a relationship between different materials property variables. This object can be directly manipulated and stores relavent metadata available as direct attributes.

- All Models are imported as classes at runtime.
- A Model class must be instantiated to be used at runtime.

In [1]:
from propnet.core.graph import Graph
g = Graph()
model = g.get_models()['refractive_indexfrom_rel_perm']
print(model.description)
print(model.name)
print()
print(model.equations)

Propnet is not intended for public use at this time. Functionality might change.

The refractive index gives the factor by which the speed of light is reduced in
a medium.  Likewise, modeling the induced magnetic and electric dipoles as linear
within a material, a relative spatial electrical permittivity and relative spatial
magnetic permeability arise from consideration of the total electrical and magnetic
fields.

From the Maxwell Relations, the index of refraction is equal to the geometric mean
of the relative permittivity and the relative permeability.

refractive_indexfrom_rel_perm

['n = sqrt(Ur*Er)']


## Model Class Structure - Basic Overview

The Model class is a generally-defined interface, and subclasses may alter many aspects of its underlying functionality.


Most Model objects will contain equations, symbols, and connections attributes. These define the core functionality of the model:

The equations attribute will contain a list of sympy-parsable expressions. These expressions imply trivial equations such that the expression is equal to zero. 
(ie. x - 4 for corresponding equation x = 4)

In [2]:
print(model.equations)

['n = sqrt(Ur*Er)']


The symbol_mapping attribute maps the symbols (ie. n) used in the equations to Symbol objects (ie. refractive index) used in the Property Network.

In [3]:
print(model.symbol_property_map)

{'Ur': 'relative_permittivity', 'Er': 'relative_permeability', 'n': 'refractive_index'}


The connections attribute shows what outputs can be generated from a set of inputs, following a standard format.

In [4]:
print(model.connections)

[{'inputs': ['n', 'Ur'], 'outputs': ['Er'], '_lambdas': {'Er': <function <lambda> at 0x1213b0840>}}, {'inputs': ['Er', 'n'], 'outputs': ['Ur'], '_lambdas': {'Ur': <function <lambda> at 0x12142f620>}}, {'inputs': ['Er', 'Ur'], 'outputs': ['n'], '_lambdas': {'n': <function <lambda> at 0x1214068c8>}}]


A Model can be evaluated to generate outputs if given a complete set of inputs.

Given the relative permeability and permittivity, the Refractive Index From Relative Permeability model can correctly calculate the index of refraction.

This is given by 'n' in the dictionary below.

In [5]:
model.evaluate({'Ur': 0.54, 'Er': 3.9})

{'refractive_index': <refractive_index, 1.4512063946937388 dimensionless, None>,
 'successful': True}

# Loading Materials Data

Material properties can be loaded in from the Materials Project so they don't need to be defined and added manually.

Accessing Materials Project data requires an API key. You must input your own API key below to run the sample. You can locate your api key by logging into materialsproject.org and visiting the dashboard.

In [7]:
from propnet.ext.matproj import *
my_api_key = None
mpr = MPRester(my_api_key)
silica = mpr.get_properties_for_mpid('mp-546794')
print(silica)

{'material_id': 'mp-546794', 'band_gap.search_gap.band_gap': 5.703399999999999, 'computed_entry': ComputedEntry mp-546794 - Si16 O32
Energy = -379.9357
Correction = -22.4733
Parameters:
run_type = GGA
is_hubbard = False
pseudo_potential = {'functional': 'PBE', 'labels': ['Si', 'O'], 'pot_type': 'paw'}
hubbards = {}
potcar_symbols = ['PBE Si', 'PBE O']
oxide_type = oxide
Data:
oxide_type = oxide, 'diel.n': 1.4782042033043565, 'diel.poly_total': 3.8824606666666672, 'diel.e_total': [[3.827947000000001, 0.0, 0.0], [0.0, 3.827947000000001, 1.9736472213591792e-17], [0.0, 1.9736472213591792e-17, 3.9914880000000004]], 'pretty_formula': 'SiO2', 'e_above_hull': 0, 'elasticity.elastic_tensor': [[91.0, -38.0, 17.0, 0.0, 0.0, 0.0], [-38.0, 91.0, 17.0, 0.0, 0.0, 0.0], [17.0, 17.0, 86.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 32.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 32.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 30.0]], 'formation_energy_per_atom': -3.2847129075, 'magnetic_type': 'NM', 'oxide_type': 'oxide', 'piezo.pie

# Working with Units

## Symbol Objects

No units need to be specified when defining a Quantity object or plugging values into Models. In the case that no units are supplied, default units are inferred based on those provided in the corresponding Symbol object.

For instance, youngs_modulus is a Symbol with the default units of gigapascal. 
In the example above:

           youngs_modulus = Symbol('youngs_modulus', 200, [])
           
youngs_modulus automatically has the value 200 gigapascals despite no units being provided.

It is possible to provide your own units by working with the <b>unit registry</b> (ureg).<br></br>
The examples below demonstrate this capability using the method: 
            
            ureg.Quantity(value, unit)
            
The property network will automatically convert the units you supply into canonical units behind the scenes for use in models that use the property.

In [5]:
from propnet import ureg
from propnet.core.graph import Graph
from propnet.core.materials import Material
from propnet.core.quantity import Quantity

g = Graph()

mild_steel = Material()
yield_strength = Quantity('yield_stress', ureg.Quantity(53700, 'psi'), [])
mild_steel.add_quantity(yield_strength)
print(mild_steel)

Material: 0x1214e1dd8

	yield_stress
		...



## Models
No units need to be specified when plugging values into a Model. In the case that no units are supplied, default units are inferred based on those provided in the corresponding SymbolType objects.

            model.evaluate(dict)
            
The above method can be used to plug in raw python floats, indicating that default units are implied. 

The above method can also be used with custom units supplied to one or more of the values in the dictionary. 

In the example below we use the property network to calculate the Peierls-Nabarro Stress for a disloaction on the (111) plane in Aluminum (FCC) using custom units of psi for shear modulus.

In [1]:
from propnet import ureg
from propnet.core.graph import Graph
from propnet.core.quantity import Quantity

g = Graph()
model = g.get_models()['peierls_stress']

G = ureg.Quantity(3.9*10**6, 'psi')
a = 4.046 / 3**(1/2)  # Angstroms automatically assumed by the model.
b = 4.046 / 2**(1/2)  # Angstroms automatically assumed by the model.
nu = 0.33

tau = model.evaluate({'G': G.magnitude, 'a': a, 'b': b, 'nu': nu})
print(tau)

Propnet is not intended for public use at this time. Functionality might change.

{'peierls_stress': <peierls_stress, 1843.6077548007825 gigapascal, None>, 'successful': True}


## Converting Units
Given you have an output with units, it is straightforward to convert between units using Pint.

In [2]:
# Converting to megapascals.
print(tau['peierls_stress'].to('megapascal'))

<peierls_stress, 1843607.7548007825 megapascal, None>


# Creating Custom Models and Properties

The property network comes with many different models and properties pre-loaded and ready for use.

To add additional properties to the project can be accomplished in two different forms:

- Defining additional .yaml and .py files in the propnet.models and propnet.symbols folders respectively. For these changes to take effect, the property network must be re-loaded.

- Defining additional Model and SymbolType classes at runtime.

Both approaches will be detailed below.

## Defining new model files:
Models require a model_name.yaml and a model_name.py file to be defined in the propnet/models folder.

The syntax for a model.yaml definition is as follows:

<table style="width:100%">
  <tr>
    <th>Field Name</th>
    <th>Field Format</th> 
    <th>Field Description</th>
  </tr>
  <tr>
    <td>title</td>
    <td>string</td> 
    <td>title of the model</td>
  </tr>
  <tr>
    <td>tags</td>
    <td>list of strings</td> 
    <td>any user-specific tags applying to the model</td>
  </tr>
  <tr>
    <td>references</td>
    <td>list of strings</td> 
    <td>any urls for citing the model</td>
  </tr>
  <tr>
    <td>symbol_mapping</td>
    <td>dictionary, string keys, string values</td> 
    <td>mapping from symbols used in the model to SymbolType names used in the property network</td>
  </tr>
  <tr>
    <td>connections</td>
    <td>list of dictionaries with keys: 'inputs', 'outputs' mapping to lists of symbol strings used in the model</td> 
    <td>datastrucutre used to determine the set of all outputs that can be generated from a set of inputs</td>
  </tr>
  <tr>
    <td>equations</td>
    <td>list of strings</td> 
    <td>list of equations that create the model, defined as the value of the expression equals zero</td>
  </tr>
</table>

By default the python file can be a simple class declaration:

            class model_name(AbstractModel):
                pass
                
Methods of the AbstractModel class can be overridden to allow for custom model evaluation() or constraint handling (see below).

## Defining new Symbol files:
Symbols require a symbol.yaml file to be defined in the propnet/symbols folder.
The syntax for a symbol_type.yaml definition is as follows:
<table style="width:100%">
  <tr>
    <th>Field Name</th>
    <th>Field Format</th> 
    <th>Field Description</th>
  </tr>
  <tr>
    <td>name</td>
    <td>string</td> 
    <td>name of the property, must match the file name</td>
  </tr>
  <tr>
    <td>units</td>
    <td>nested list: [1.0, [[unit name string, unit_power], ...]</td> 
    <td>the units used to represent this SymbolType, adjacent entries in the list indicate multiplication.</td>
  </tr>
  <tr>
    <td>display_names</td>
    <td>list of strings</td> 
    <td>any strings useful for representing this SymbolType</td>
  </tr>
  <tr>
    <td>display_symbols</td>
    <td>list of strings</td> 
    <td>any short string symbols useful for representing this SymbolType</td>
  </tr>
  <tr>
    <td>dimension</td>
    <td>list of numbers</td> 
    <td>datastructure to give the length of each dimension in a higher order tensor representation of the property, 1 for a scalar value.</td>
  </tr>
  <tr>
    <td>comment</td>
    <td>string</td> 
    <td>any comments pertinent to the SymbolType</td>
  </tr>
  <tr>
    <td>category</td>
    <td>string</td> 
    <td>whether this is a property, object, or condition</td>
  </tr>
</table>

# A Grand Example

In [6]:
silicon = Material()
silicon.add_quantity(Quantity('temperature', 300, []))
silicon.add_quantity(Quantity('band_gap', 1.12, []))
silicon.add_quantity(Quantity('electrical_conductivity', 4.34E-4, []))

aluminum = Material()
aluminum.add_quantity(Quantity('temperature', 300, []))
aluminum.add_quantity(Quantity('band_gap', 0, []))
aluminum.add_quantity(Quantity('electrical_conductivity', 3.69E7, []))

g = Graph()
silicon = g.evaluate(silicon)
aluminum = g.evaluate(aluminum)

print("Silicon's Graph")
print(silicon)
print()
print("Aluminum's Graph")
print(aluminum)

Silicon's Graph
Material: 0x122bafef0

	temperature
		<temperature, 300 kelvin, None>

	band_gap
		<band_gap, 1.12 electron_volt, None>

	electrical_conductivity
		...

	band_gap_pbe
		...
		...

	refractive_index
		...
		...
		...
		...
		...

	band_gap_gw
		...

	electrical_resistivity
		...

	is_metallic
		<is_metallic, 0 dimensionless, None>

	electronic_thermal_conductivity
		...


Aluminum's Graph
Material: 0x122bafcc0

	temperature
		<temperature, 300 kelvin, None>

	band_gap
		<band_gap, 0 electron_volt, None>

	electrical_conductivity
		...

	refractive_index
		...
		<refractive_index, 4.16 dimensionless, None>

	band_gap_gw
		...

	electrical_resistivity
		...

	is_metallic
		<is_metallic, 1 dimensionless, None>

	electronic_thermal_conductivity
		...

